In [1]:
from wallet import Wallet
from accounts import SHMAccount

### Wallets


- CorporateSuperWallet is the main wallet. I AM THINKING TO MAKE MAIN WALLET A COMPANY WALLET INSTEAD, WHERE THERE IS NO PRIVATE KEYS.
- CorporateWallet: cosigners at the top corporate level.
- StoreWallet: responsible for only address generation for payments in each store.
- DailySafeWallet: Multisignature single-herarchical wallet. Manageble (see balance and create transactions) by everybody but AssistantManagerWallet who only can authorize in emergency events.
- WeeklySafeWallet: same as DailySafeWallet with the exception that no AssistantManagerWallet is part of this wallet.
- CorporateSafeWallet: Multisignature fully-herarchical wallet. Supported on the non_hardened keys within tge year account which is a hardened derived account.
- ManagerWallet: takes part in the DailySafeWallet and the WeeklySafeWallet.
- AssistantManagerWallet: only a cosigner wallet. This is the most basic and simple wallet type.


In [2]:
class CorporateSuperWallet(Wallet):
    
    def create_store_account(self, store_code):
        """
        Returns the master private key of the store with index 'store_code'.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        return 
    
    def request_pubkeys_daily_safe(self,store_code, m=2,n=6):
        """
        Returns the message to start requesting the public keys necessary to setup a
        daily safe.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        #save in the database the unique_id, public keys, m, n, and the store_code.
        return 
    
    def create_daily_safe_wallet(self,master_pubkey, public_key_list, m=2, n=6,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        shm_account = SHMAccount(m,n, daily_safes_acc.xtended_public_key, public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_weekly_safe_wallet(self,master_pubkey, public_key_list, m=2, n=5,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        path = "m/44H/0H/2H"
        weekly_safes_acc = self.get_child_from_path(path)
        shm_account = SHMAccount(m,n, weekly_safes_acc.xtended_public_key, public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_corporate_safe(self, index):
        return 
    
    def get_store_deposit_account(self, store_index):
        return

In [3]:
class StoreWallet(Wallet):
    
     def create_daily_safe(self, date_index, public_key_list,m=2,n=6):
        """
        returns a multisignature account with the public keys and the index specified.
        date_index: Integer. Must follow YYMMDD format.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length.
        """
        return 
    

In [5]:
class ManagerWallet(Wallet):
    def accept_daily_safe_wallet(self,_privkey,invite):
        """
        accepts the invitation from a SuperCorporateWallet to participate in a daily-safe wallet.
        The acceptance must be through a private key that matches one of the public keys that are
        part of the SHM wallet passed.
        _prvkey: PrivateKey object. Must be able to generate one of the public keys of the wallet.
        invite: SHMAccount object. The wallet that we are going to be part of.
        """
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        shm_account = SHMAccount(invite.m,invite.n, invite.xtended_pubkey, invite.public_key_list, 
                                 daily_safes_acc.private_key_obj,invite.addr_type,invite.testnet, 
                                 invite.segwit)
        return shm_account
        